In [1]:
include("../LiPoSID.jl")

include("CtrlSID.jl")


using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
│ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.
└ @ nothing nothing:984


In [2]:
#using Plots
#using Optim
#using Random

In [12]:
function det_2x2(matrix)
    # Ensure the matrix is 2x2
    if size(matrix) != (2, 2)
        throw(ArgumentError("Matrix must be 2x2"))
    end

    # Calculate the determinant
    return matrix[1, 1] * matrix[2, 2] - matrix[1, 2] * matrix[2, 1]
end

function sqrt_32th(x)
    #16th: 1/4 + 2(x-1/16) - 8(x-1/16)^2 + 64(x-1/16)^3 - 160(x-1/16)^4 + 112(x-1/16)^5
    1/(4√2) + 2√2(x-1/32) - 16√2(x-1/32)^2 + 512√2/6(x-1/32)^3 - 8192√2/24(x-1/32)^4 
    + 131072√2/120(x-1/32)^5 - 2097152√2/720(x-1/32)^6 
    #+ 33554432√2/factorial(7)*(x-1/32)^7 - 536870912√2/factorial(8)*(x-1/32)^8
end

function sqrt_cheb(x)
    -23039.545043081274*x^4+3836.399516089045*x^3-242.53552064367273*x^2+9.38274438236273*x+0.02531162814470008
end

function fidelity_norm(ρ, σ)
    tr(ρ * σ)+2*sqrt_cheb(det_2x2(ρ)*det_2x2(σ))
end

function kossak_infid_cntrl_obj(ρ, t, H0ˢʸᵐᵇ, Vˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ, ω)

    Hˢʸᵐᵇ(t) = H0ˢʸᵐᵇ + Vˢʸᵐᵇ * sin(ω * t)

    function Dc(ρ, t)
        U = (Hˢʸᵐᵇ(t)*ρ - ρ*Hˢʸᵐᵇ(t))/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        ρᵢᵃᵖʳᴼˣ = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i], t[i])+4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        obj += 1 - fidelity_norm(ρ[i], ρᵢᵃᵖʳᴼˣ)
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

kossak_infid_cntrl_obj (generic function with 1 method)

In [13]:
function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    Vˢʸᵐᵇ = [0 0
             0 0]

    ω = 0

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_infid_cntrl_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Vˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ, ω)
    polyE = kossak_infid_cntrl_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Vˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ, ω)
    polyX = kossak_infid_cntrl_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Vˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ, ω)
    polyY = kossak_infid_cntrl_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Vˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ, ω)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

function kossak_time_evolution(ρₒ, tᵉⁿᵈ, dt, H, C, fᴼᴺᴮ)

    V = [0 0
         0 0]

    ω = 0

    t, ρ = CtrlSID.kossak_cntrl_time_evolution(ρₒ, tᵉⁿᵈ, dt, H, V, C, fᴼᴺᴮ, ω)

    return t, ρ
    
end


kossak_time_evolution (generic function with 1 method)

### Kossakowski objective for two-level system


For the two level system we can use theorem originally proved by Kossakowski:

Kossakowski, Bull. Acad. Pol. Sci. Ser. Math. Astr. Phys. 21, 649 (1973).

and available also in classic GKS paper:

Completely positive dynamical semigroups of Nlevel systems
Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan
Citation: J. Math. Phys. 17, 821 (1976); doi: 10.1063/1.522979

Theorem 3.1. (two-level system)

A linear operator $L: M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$L = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where

(i) $H = \sum_1^3 h_i f_i, \quad h_i \in \R$

(ii) $f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$


(iii) $\{C_{ij}\}= \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} =$

$=\begin{pmatrix} 
-\gamma_1 + \gamma_2 + \gamma_3   & - i a_3   &  i a_2 \\
i a_3                 & \gamma_1 - \gamma_2 + \gamma_3 & -i a_1 \\
-i a_2                & i a_1      & \gamma_1 + \gamma_2 - \gamma_3 
\end{pmatrix}$

To write a Hamiltonian $H$ and than objective function we can use the set of orthonormal operators $f_j$

We use Pauli matrices $\sigma_x, \sigma_y, \sigma_z$ to choose 3 orthonormal $f_i$ for the two-level system to fulfill:

$f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$

In [4]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

3-element Vector{Matrix{ComplexF64}}:
 [0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.0 + 0.0im]
 [0.0 + 0.0im 0.0 + 0.5im; 0.0 - 0.5im 0.0 + 0.0im]
 [0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.5 + 0.0im]

In [5]:
@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

H0ˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

H0ᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(H0ᴷˢʸᵐᵇ) == 0
@assert H0ᴷˢʸᵐᵇ == H0ˢʸᵐᵇ

Hˢʸᵐᵇ = H0ˢʸᵐᵇ 

#@polyvar d_Re d_Im

#Vˢʸᵐᵇ =  [ 0             d_Re+im*d_Im
#           d_Re-im*d_Im  0.           ]    

@polyvar γ[1:3]
@polyvar a[1:3]

Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]
           im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

κ₁ = -γ[1]+γ[2]+γ[3]
κ₂ =  γ[1]-γ[2]+γ[3]
κ₃ =  γ[1]+γ[2]-γ[3]

constr1 = κ₁ + κ₂ + κ₃  
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - - a[1]^2 - a[2]^2 - a[3]^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2 


constraints = [γ[1], γ[2], γ[3], constr1, constr2, constr3]

6-element Vector{Polynomial{true, Int64}}:
 γ₁
 γ₂
 γ₃
 γ₁ + γ₂ + γ₃
 -γ₁² + 2γ₁γ₂ + 2γ₁γ₃ - γ₂² + 2γ₂γ₃ - γ₃² + a₁² - a₂² - a₃²
 -γ₁³ + γ₁²γ₂ + γ₁²γ₃ + γ₁γ₂² - 2γ₁γ₂γ₃ + γ₁γ₃² + γ₁a₁² - γ₁a₂² - γ₁a₃² - γ₂³ + γ₂²γ₃ + γ₂γ₃² - γ₂a₁² + γ₂a₂² - γ₂a₃² - γ₃³ - γ₃a₁² - γ₃a₂² + γ₃a₃²

In [6]:

function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end


function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

In [7]:
function TrDist(ρ₁, ρ₂)
    A = ρ₁-ρ₂
    D = tr(sqrt(A'*A))/2
    if abs(imag(D))>1e-6
        throw(DomainError(D, "Trace distance is complex number"))
    else
        return(real(D))
    end
end 

TrDist (generic function with 1 method)

In [8]:
#tests_dir = "../TESTS/"
tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

In [14]:
println(" SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

tests_data_file_name = "POP_NEW_TSSOS_INFID_KOS_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []

for γᵢ in γ

    println("γ =  "*γᵢ)

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

    sol, opt_val, status = CtrlSID.cs_tssos(polyGEXY, constraints)
    print(" status:", status)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol); Cˢⁱᵈ = subs(Cˢʸᵐᵇ, sol)
    epsilon = subs(ϵ, sol)

    push!(Epsilons, epsilon)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   
        γ_group["epsilon"] = convert(Float64, epsilon)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)

    end

    println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        ρₒ = ρₛ[1]
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        tˢⁱᵈ, ρˢⁱᵈ  = kossak_time_evolution(ρₒ, tᵉⁿᵈ, dt, Hˢⁱᵈ, Cˢⁱᵈ, fᴷᴼᴺᴮ)
        bˢⁱᵈ = LiPoSID.bloch(ρˢⁱᵈ)

        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
            init_state_group["time"] = tᵗˢᵗ
        end
        
        FminState = minimum(F)
        FmedianState = mean(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Energy"] = convert.(Float64,Epsilons)
end


println(tests_data_file_name)

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective
γ =  0.079477
TSSOS failed: InterruptException()
 status:Error

MethodError: MethodError: no method matching subs(::Matrix{Polynomial{true, ComplexF64}}, ::Pair{Vector{PolyVar{true}}, Nothing})
Closest candidates are:
  subs(::Any, !Matched::Union{Pair{<:AbstractVariable}, Pair{<:Tuple{Vararg{AbstractVariable, N}}, <:Tuple{Vararg{Any, N}}} where N, Pair{<:Tuple{Vararg{AbstractVariable}}, <:AbstractVector}, Pair{<:AbstractVector{<:AbstractMonomialLike}, <:AbstractVector}, Pair{<:AbstractVector{<:AbstractMonomialLike}, <:Tuple}}...) at ~/.julia/packages/MultivariatePolynomials/sWAOE/src/substitution.jl:113

In [10]:
println(tests_data_file_name)

POP_NEW_TSSOS_KOS_2024-Jun-17_at_07-56.h5
